In [297]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

desc_col = {}

description_df = pd.read_excel('src/New_query_Description.xlsx',header=1)
description_df = description_df[['Column','Description']].dropna()
x = 0
for i, row in description_df.iterrows():
    desc_col[x] = row['Column']
    x += 1

In [298]:
descriptions = description_df['Description'].tolist()

In [299]:
pointx_cols = pd.read_csv('src/pointx_fbs_rpt_dly.csv').columns.to_list()
assert len(pointx_cols) == len(desc_col), "Length of columns in Pointx table and Descriptions are not equal"

In [300]:
compare_df = pd.read_csv('src/compare_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()

In [302]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_col_max_score(question,description_df=description_df,model=model):
    q_emb = model.encode(question)
    scores = [float(util.cos_sim(q_emb, model.encode(des))) for des in description_df['Description'].to_list()]
    return int(np.argmax(scores))

In [304]:
for q in questions[:5]:
    print(q)
    print(desc_col[get_col_max_score(q)],'\n')

How many daily active users each day?
deal_type 

How many monthly active users each month?
privacy_info_ads_storage 

What is the average number of daily active users last 7 days?
user_ltv_revenue 

What is the mean number of daily active users from last 30 days?
user_ltv_revenue 

When was the last time each user was active?
user_properties_ga_session_number_set_timestamp_micros 



In [150]:
import sqlparse

def extract_columns_and_conditions_per_line(sql_query):
    def get_column_names(token):
        if isinstance(token, sqlparse.sql.IdentifierList):
            return [str(ident) for ident in token.get_identifiers()]
        return [str(token)]
    
    def process_condition(token):
        column_names_condition = []
        for tok in token.tokens:
            if isinstance(tok, sqlparse.sql.IdentifierList):
                column_names_condition.extend(get_column_names(tok))
            elif isinstance(tok, sqlparse.sql.Identifier):
                column_names_condition.extend(get_column_names(tok))
        return column_names_condition

    def process_line(line):
        column_names_condition = []
        parsed_line = sqlparse.parse(line)[0]
        for token in parsed_line.tokens:
            if isinstance(token, sqlparse.sql.Parenthesis):
                # column_names_condition.extend(extract_columns_and_conditions_per_line(token))
                print(str(token).split(''))
            if isinstance(token, sqlparse.sql.Where):
                column_names_condition.extend(process_condition(token))
            elif isinstance(token, sqlparse.sql.IdentifierList) and 'SELECT' in line.upper():
                column_names_condition.extend(get_column_names(token))
            elif isinstance(token, sqlparse.sql.Identifier):
                column_names_condition.extend(get_column_names(token))
            
        return column_names_condition

    lines = sql_query.splitlines()
    result = []
    for line in lines:
        column_names_condition = process_line(line)
        result.append({
            "ColumnNamesCondition": column_names_condition
        })
    return result

# Example SQL query with multiple lines
s = """SELECT name, email
     FROM user
     WHERE email IS NOT NULL
     AND id IN (SELECT cID FROM customer WHERE points > 5)
"""

# Call the function to extract column names and conditions for each line
results = extract_columns_and_conditions_per_line(s)
for idx, result in enumerate(results, start=1):
    print(f"Line {idx}:")
    print("Column Names (CONDITION):", result["ColumnNamesCondition"])
    print()


['', 'SELECT cID FROM customer WHERE points > 5)']
Line 1:
Column Names (CONDITION): ['name', 'email']

Line 2:
Column Names (CONDITION): []

Line 3:
Column Names (CONDITION): ['email']

Line 4:
Column Names (CONDITION): ['id']



In [180]:
import sqlparse
def queries(d):
  if type(d) != sqlparse.sql.Token:
     paren = isinstance(d, sqlparse.sql.Parenthesis)
    #  print(d if not paren else d[1:-1])
     v = [queries(i) for i in (d if not paren else d[1:-1])]
     subseq, qrs = ''.join(str(i[0]) for i in v), [x for _, y in v for x in y]
     if [*d][paren].value == 'SELECT':
        return '<subquery>', [subseq]+qrs
     return subseq, qrs
  return d, []

s="""SELECT name, email
     FROM (SELECT * FROM user WHERE email IS NOT NULL)
     WHERE id IN (SELECT cID FROM customer WHERE points > 5)
"""
_, subqueries = queries(sqlparse.parse(s)[0])
subqueries

['SELECT name, email\n     FROM <subquery>\n     WHERE id IN <subquery>\n',
 'SELECT * FROM user WHERE email IS NOT NULL',
 'SELECT cID FROM customer WHERE points > 5']

In [176]:
_

'<subquery>'

Cosine-Similarity: tensor([[0.8958]])
